# Kubernetes demo

# Description
In this demo I'll show you how to create a Postgres cluster with CloudNativePG kubernetes operator. The features that I want to show you are:
- Kubernetes plugin install
- CloudNativePG operator install
- Postgres cluster install
- Insert data in the cluster
- Switchover (promote)
- Failover
- Backup
- Recovery
- Rolling updates (minor and major)

# Prerequisites
- K8s environment
- Tested with Docker Desktop with Kubernetes on Mac 


## Demo prep

### Find IP address of this machine

In [ ]:
ipconfig getifaddr en0
ipconfig getifaddr en6

Enter this IP address in mycluster-upgrade.yaml

### Start Minio

In [1]:
kubectl create secret generic minio-creds \
  --from-literal=MINIO_ACCESS_KEY=admin \
  --from-literal=MINIO_SECRET_KEY=password

docker run -p 9000:9000 -p 9001:9001 \
           -e MINIO_ROOT_USER=admin \
           -e MINIO_ROOT_PASSWORD=password \
           -d --rm \
           minio/minio server /data \
           --console-address ":9001"

error: failed to create secret Post "https://kubernetes.docker.internal:6443/api/v1/namespaces/default/secrets?fieldManager=kubectl-create&fieldValidation=Strict": tls: failed to verify certificate: x509: certificate signed by unknown authority (possibly because of "crypto/rsa: verification error" while trying to verify candidate authority certificate "kubernetes")
114506d66c10a5dcedc2cc85e502de6687777e35e3a060a7da9f8ab246998d81


## Configuration

### Install plugin

In [ ]:
curl -sSfL https://github.com/cloudnative-pg/cloudnative-pg/raw/main/hack/install-cnpg-plugin.sh | sh -s -- -b /usr/local/bin

### Install operator

In [ ]:
version1=`kubectl-cnpg version | awk '{ print $2 }' | awk -F":" '{ print $2}'`
version2=${version1%??}
kubectl apply -f https://raw.githubusercontent.com/cloudnative-pg/cloudnative-pg/release-$version2/releases/cnpg-$version1.yaml

### Check operator installed

In [ ]:
kubectl get deploy -n cnpg-system cnpg-controller-manager -o wide

## Create cluster

### Install cluster

In [ ]:
cat mycluster.yaml

In [ ]:
kubectl apply -f mycluster.yaml

In [ ]:
kubectl get pods

### Show status

`watch -c -n 1 kubectl-cnpg status mycluster`

### Insert data

In [ ]:
cat sqltest.sql

In [ ]:
kubectl exec -i mycluster-1 -- psql < sqltest.sql

### Check data

In [ ]:
kubectl exec -it mycluster-2 -- psql -c "select * from test limit 15;"

In [ ]:
kubectl exec -it mycluster-2 -- psql -c "select count(*) from test;"

## Promotion 

### Promotion of node 2 to Primary

In [ ]:
kubectl-cnpg promote mycluster mycluster-2

## In-place rolling upgrade minor version

### Update from Postgres 14.2 to 14.5

In [ ]:
cat mycluster-upgrade.yaml

In [ ]:
kubectl apply -f mycluster-upgrade.yaml

## Backup & restore

### Open the Minio console

In [ ]:
open http://localhost:9001

### Backup cluster

In [ ]:
cat backup.yaml

In [ ]:
kubectl apply -f backup.yaml

### Verify backup

In [ ]:
kubectl describe backup mybackup

### Restore to new cluster

In [ ]:
cat restore.yaml

In [ ]:
kubectl exec -it mycluster-2 -- psql -c "select pg_switch_wal();"
kubectl apply -f restore.yaml

### Verify restore

`watch -c -n 1 kubectl-cnpg status mycluster-restore`

In [ ]:
kubectl exec -it mycluster-restore-1 -- psql -c "select * from test limit 15;"

In [ ]:
kubectl exec -it mycluster-restore-1 -- psql -c "select count(*) from test;"

## Failover 

### Delete Primary pod

In [ ]:
kubectl delete pod mycluster-2 --force

`watch -c -n 1 kubectl-cnpg status mycluster`

## Migration major version

### Create Postgres 13 cluster

In [ ]:
kubectl apply -f cluster-example-13.yaml

`watch -c -n 1 kubectl-cnpg status cluster-example-13`

### Add data to cluster

In [ ]:
kubectl exec -it cluster-example-13-1 -- psql app < sqltest.sql

### Verify inserted data

In [ ]:
kubectl exec -it cluster-example-13-1 -- psql app -c "select * from test limit 15;"

In [ ]:
kubectl exec -it cluster-example-13-1 -- psql app -c "select count(*) from test;" app

### Create new Postgres 15 cluster based on Postgres 13 cluster

In [ ]:
kubectl apply -f cluster-example-upgrade-13-to-15.yaml

### Verify data in new cluster

In [ ]:
kubectl exec -it cluster-example-15-1 -- psql app -c "select * from test limit 15;"

In [ ]:
kubectl exec -it cluster-example-15-1 -- psql app -c "select count(*) from test;"

# End of demo

## How to deploy and access the Kubernetes Dashboard

In [ ]:
./dashboard_install.sh

# Links
[https://github.com/minio/minio](https://github.com/minio/minio)

[https://cloudnative-pg.io](https://cloudnative-pg.io)

# Cleanup

In [2]:
kubectl delete backup mybackup
kubectl delete deployments.apps -n cnpg-system cnpg-controller-manager
version1=`kubectl-cnpg version | awk '{ print $2 }' | awk -F":" '{ print $2}'`
version2=${version%??}
kubectl delete -f https://raw.githubusercontent.com/cloudnative-pg/cloudnative-pg/release-$version2/releases/cnpg-$version1.yaml
kubectl delete secret minio-creds
docker ps | grep minio | awk '{print $1}' | xargs -I % docker stop %
kubectl delete cluster mycluster-13
kubectl delete cluster mycluster-15
kubectl delete cluster mycluster
kubectl delete cluster mycluster-restore
docker pull ghcr.io/cloudnative-pg/postgresql:13
docker pull ghcr.io/cloudnative-pg/postgresql:14.2
docker pull ghcr.io/cloudnative-pg/postgresql:14.5
docker pull ghcr.io/cloudnative-pg/postgresql:15.1

error: the server doesn't have a resource type "backup"
Error from server (NotFound): deployments.apps "cnpg-controller-manager" not found
error: unable to read URL "https://raw.githubusercontent.com/cloudnative-pg/cloudnative-pg/release-/releases/cnpg-1.21.1.yaml", server reported 404 Not Found, status code=404
Error from server (NotFound): secrets "minio-creds" not found
114506d66c10
error: the server doesn't have a resource type "cluster"
error: the server doesn't have a resource type "cluster"
error: the server doesn't have a resource type "cluster"
error: the server doesn't have a resource type "cluster"
13: Pulling from cloudnative-pg/postgresql
Digest: sha256:677219dd03bf3bbff1ec85fb9a6da6e2f06f44a2156bb9afae9c803ab7191716
Status: Image is up to date for ghcr.io/cloudnative-pg/postgresql:13
ghcr.io/cloudnative-pg/postgresql:13

What's Next?
  View a summary of image vulnerabilities and recommendations → docker scout quickview ghcr.io/cloudnative-pg/postgresql:13
14.2: Pulling fr

## Uninstall Kubernetes Dashboard

In [ ]:
./dashboard_uninstall.sh